In [2]:
import sys
sys.path.append(r"G:\project\map\wsgi\mindmap\oversea")

In [3]:
urls = ['http://cs.mines.edu/CS-Faculty',
            'http://science.iit.edu/computer-science/people/faculty',
            'http://cms.bsu.edu/academics/collegesanddepartments/computerscience/facultyandstaff/faculty',
            'http://www.memphis.edu/cs/people/index.php',
            'http://www.smu.edu/Lyle/Departments/CSE/People/Faculty',
            'https://www.csuohio.edu/engineering/eecs/faculty-staff',
            'http://computerscience.engineering.unt.edu/content/faculty',
            'https://www.odu.edu/compsci/research',
            'http://www.cs.ucf.edu/people/index.php',
            'https://www.binghamton.edu/cs/people/faculty-and-staff.html',
            'http://www.lsu.edu/eng/ece/people/index.php',
        'http://www.uwyo.edu/cosc/cosc-directory/',
        'http://www.cs.siu.edu/faculty-staff/continuing_faculty.php',
        'https://www.uml.edu/Sciences/computer-science/faculty/',
            ]
examples = ['https://inside.mines.edu/CS-Faculty-and-Staff/TracyCamp',
            'http://science.iit.edu/people/faculty/eunice-santos',
            'http://cms.bsu.edu/academics/collegesanddepartments/computerscience/facultyandstaff/faculty/buispaul',
            'http://www.memphis.edu/cs/people/faculty_pages/william-baggett.php',
            'http://www.smu.edu/Lyle/Departments/CSE/People/Faculty/ThorntonMitchell',
            'https://www.csuohio.edu/engineering/charles-k-alexander-professor',
            'http://www.cse.unt.edu/~rakl',
            'https://www.odu.edu/directory/people/a/achernik',
            'http://www.cs.ucf.edu/~bagci',
            'https://www.binghamton.edu/cs/people/kchiu.html',
            'http://www.lsu.edu/eng/ece/people/Faculty/brown.php',
            'http://www.uwyo.edu/cosc/cosc-directory/jlc/index.html',
            'http://www.cs.siu.edu/~abosu',
            'https://www.uml.edu/Sciences/computer-science/faculty/levkowitz-haim.aspx'
            ]

url_check = {'http://cs.mines.edu/CS-Faculty': 15,
             'http://science.iit.edu/computer-science/people/faculty': 52,
             'http://cms.bsu.edu/academics/collegesanddepartments/computerscience/facultyandstaff/faculty': 19,
             'http://www.memphis.edu/cs/people/index.php': 16,
             'http://www.smu.edu/Lyle/Departments/CSE/People/Faculty': 17,
             'https://www.csuohio.edu/engineering/eecs/faculty-staff': 32,
             'http://computerscience.engineering.unt.edu/content/faculty': 34,
             'https://www.odu.edu/compsci/research': 15,
             'http://www.cs.ucf.edu/people/index.php': 54,
             'https://www.binghamton.edu/cs/people/faculty-and-staff.html': 33,
             'http://www.lsu.edu/eng/ece/people/index.php': 34,
             'http://www.uwyo.edu/cosc/cosc-directory/': 11,
             'http://www.cs.siu.edu/faculty-staff/continuing_faculty.php': 10,
             'https://www.uml.edu/Sciences/computer-science/faculty/': 18,
             
             }

In [20]:
import crawler
reload(crawler)


<module 'crawler' from 'G:\project\map\wsgi\mindmap\oversea\crawler.pyc'>

In [21]:
def test_single(i):
    directory_url = urls[i]
    example = examples[i]
    print i, directory_url #, example
    c = crawler.ResearchCrawler(directory_url, example)
    count, l = c.crawl_faculty_list(directory_url, example)
    print count, url_check[directory_url]
    #for e in l:
        #print e["href"]
    assert url_check[directory_url] <= count, " not beyond"
    for e in l[:3]:
        print l.index(e), e.get('href')
        # print e.get_text()
        x = c.dive_into_page(e, False)

        
        #print x['name']
        # print x['tags']
        print x['website']
crawler.debug_level = 1
test_single(13)

13 https://www.uml.edu/Sciences/computer-science/faculty/
https://www.uml.edu/Sciences/computer-science/faculty/ https://www.uml.edu uml
18 18
0 https://www.uml.edu/Sciences/computer-science/faculty/levkowitz-haim.aspx
 name is Haim Levkowitz, Ph.D.
stop word :[u'http', u'https', u'www', u'com', u'lab', u'calendar', u'staff', u'publications', u'military', u'international', u'teaching', u'people', u'faculty', u'student', u'college', u'professor', u'edu', u'lecturer', u'affiliate', u'associate', u'adjunct', u'visiting', u'assistant', u'questions?', u'aspx', u'html', 'https', 'www', 'uml', 'edu', 'Sciences', 'computer', 'science', 'faculty']
potential name: [u'levkowitz', u'haim', 'personal']
http://www.cs.uml.edu/~haim/
1 https://www.uml.edu/Sciences/computer-science/faculty/kim-byung-guk.aspx
 name is Byung Guk Kim, Ph.D.
stop word :[u'http', u'https', u'www', u'com', u'lab', u'calendar', u'staff', u'publications', u'military', u'international', u'teaching', u'people', u'faculty', u'stu

In [20]:
crawler.debug_level = 1
for i in range(11, 12):#len(urls)):
    directory_url = urls[i]
    example = examples[i]
    print i, directory_url #, example
    count, l = c.crawl_faculty_list(directory_url, example)
    print count, url_check[directory_url]
    #for e in l:
        #print e["href"]
    assert url_check[directory_url] <= count, " not beyond"
    for e in l[:5]:
        print l.index(e), e.get('href')
        # print e.get_text()
        x = c.dive_into_page(e)

        
        # print x['name']
        # print x['tags']
        print x['website']
        # print x['position']

11 http://www.uwyo.edu/cosc/cosc-directory/
http://www.uwyo.edu/cosc/cosc-directory/ 123
http://www.uwyo.edu/cosc/cosc-directory/ 37 0 11


AssertionError:  not beyond

In [17]:
x = "The UCF Natural Language Processing Group is always looking for talented PhD and undergraduate students to join us on research projects. If you are interested in working with us, please see here."

In [6]:
import os
os.makedirs

<function os.makedirs>

In [264]:
import re
re.search("(position available|looking for [\w\s]*students)", x).group(1)

'looking for talented PhD and undergraduate students'

In [342]:

crawler.debug_level = 0
faculty_link = 'http://www.cs.ucf.edu/people/index.php'
content, soup = c.open_page(faculty_link)
# c.get_research_interests(soup, content, [], faculty_link)
node = soup.find(string=re.compile("Ulas Bagci"))
print node, node.parent.name, node.parent.get('class')
# c.get_open_position(soup, content)

Ulas Bagci span [u'facultynames', u'hide-for-medium']


In [6]:
from nameparser import HumanName

In [7]:
x = HumanName("charles-k-alexander-professor")

In [8]:
x

<HumanName : [
	title: '' 
	first: 'charles-k-alexander-professor' 
	middle: '' 
	last: '' 
	suffix: ''
	nickname: ''
]>

In [352]:
a = soup.find('a', href='http://www.cs.ucf.edu/~bagci')
la = a.find_parents()[::-1]

In [350]:
lb = node.find_parents()[::-1]

In [353]:
for i in range(len(a)):
    if la[i].name == lb[i].name:
        print la[i].name
        continue
    print la[i].name, lb[i].name
    break

[document]


In [17]:
def contain_keys(href, keys, is_name=False):
    """

    """
    if not keys:
        return False
    words = "(%s)" % '|'.join(e for e in keys)
    if is_name and re.search('%s' % words, href, re.I):
        return True
    if re.search(r'\b%s\b' % words, href, re.I):
        return True
    if re.search(r'_?%s_?' % words, href, re.I):
        return True
    return False


def format_url(href, domain, index=''):

    if href.startswith('http'):
        full_url = href
    elif href.startswith('//'):
        full_url = 'http:' + href
    elif href[0] == '/' and (len(href) == 1 or href[1] != '/'):
        full_url = domain + href
    elif href.find(domain) > -1:
        full_url = 'http://' + href
    else:
        full_url = index + href if index[-1] == '/' else index + '/' + href
    return full_url


def get_and_store_page(page_url):

    parts = page_url.split("/")[2].split(".")
    university_name = parts[-2]
    dir_name = os.path.join(os.environ.get('OPENSHIFT_PYTHON_LOG_DIR', '.'), 'data', university_name)
    if not os.path.isdir(dir_name):
        os.mkdir(dir_name)
    file_name = re.sub("[?%=]", "", dir_name+'/' + (page_url.split('/')[-1] if page_url[-1] != '/' else page_url.split('/')[-2]) + '.html')
    if os.path.isfile(file_name):
        with open(file_name) as fp:
            html = fp.read()
    else:
        try:
            r = requests.get(page_url)
            html = r.content
        except (ConnectionError, HTTPError), e:
            html = "Error at ", page_url

        with open(file_name, 'w') as fp:
            fp.write(html)
    return html


def onsocial(href):
    href = href.lower()
    for e in ['facebook', 'twitter', 'google', 'youtube', 'calendar']:
        if e in href:
            return True
    return False


def same_tags(a, b):
    if len(a) != len(b):
        return False
    for i in range(len(a)):
        if a[i] != b[i]:
            return False
    return True

In [15]:
class ResearchCrawler:
    """
    从院系的Faculty目录中爬取有内容的教授信息
    如研究兴趣、招生机会
    """

    def __init__(self):
        self.url = ""
        self.university_name = ""
        self.domain = ""
        self.example = ""
        self.key_words = None
        self.load_key()

    def crawl_faculty_list(self, directory_url, example):
        self.example = example
        self.url = directory_url
        self.university_name = re.search('(\w+).edu', self.url).group(1)
        self.domain = '/'.join(directory_url.split("/")[:3])
        self.key_words['stop_word'] += re.findall("(\w+)", directory_url)
        
        content, soup = self.open_page(directory_url)
        anchors = self.find_all_anchor(soup, self.domain, directory_url)
        # print directory_url, len(anchors)
        index = self.find_example_index(anchors, example)
        # print directory_url, len(anchors[index:]), 

        # 第一个教授主页的作用主要在这里——如果能再来一个更好
        # 求共同祖先
        count, faculty_list = self.find_faculty_list(anchors[index:], directory_url)
        return count, faculty_list
    
    def crawl_from_directory(self, directory_url, example):
        """
        从目录爬
        """

        count, faculty_list = self.crawl_faculty_list(directory_url, example)
        # assert count >= url_check[url]
        result = []
        for one in faculty_list:
            result.append(self.dive_into_page(one))
        return result

    def load_key(self):
        with open(os.path.join(os.path.dirname(__name__), 'key.json')) as fp:
            self.key_words = json.loads(fp.read(), strict=False)
        if self.key_words is None:
            return 'Error'

    def open_page(self, page_url):
        """

        """
        try:
            html = get_and_store_page(page_url)
            if html.startswith("Error at "):
                return "Error to load", None
            soup = BeautifulSoup(html, 'html.parser')

            if soup.find("noframes"):
                for e in soup.find_all("frame"):
                    if contain_keys(e.get("src"), self.key_words["div_pass"]):
                        continue
                    page_url = page_url + e.get("src") if page_url[-1] == '/' else page_url + '/' + e.get("src") 
                    html = get_and_store_page(page_url)
                    soup = BeautifulSoup(html, 'html.parser')
            return html, soup
        except Exception, e:
            traceback.print_exc()
            return "Error to load", None

    def find_all_anchor(self, soup, domain, page_url):
        """

        """
        l = soup.find_all('a')
        return l

    def filter_list(self, e):
        name = e.string
        href = e.get('href')
        base_faculty_filter = [e for e in self.example.split('/')
                               if e and not e.startswith('http') and not e.endswith('.edu')]

        if href and '~' in href:
            return False
        elif name is not None and contain_keys(name, self.key_words['site_flag']):
            return False
        elif href and href.startswith('mailto:'):
            return True
        elif not href or len(href) < 5 or base_faculty_filter[0] not in href:
            return True
        elif contain_keys(href, self.key_words['notprof']):
            return True
        elif not contain_keys(href, self.key_words['keys']):
            return True
        return False

    
    def find_example_index(self, l, a):
        for i in range(len(l)):
            href = l[i].get("href")
            if not href:
                continue
            href = format_url(l[i].get("href"), self.domain, a)
        #   print page_url, a
            if href == a:
                return i
        return -1

    def filter_research_interests(self, alist):
        """
        暂时只想到这么多条件
        """
        result = []
        for e in alist:
            if e.parent.name == 'a':
                continue
            if contain_keys(e, self.key_words['notresearch']):
                continue
            result.append(e)
        return result

    def select_line_part(self, line):
        pos = 0
        for flag in self.key_words['interest_line_mode']:
            new_pos = line.find(flag)    
            if new_pos > pos:
                pos = new_pos + len(flag)
                break
        return line[pos:]

    def replace_words(self, line):
        line = re.sub("\s+", " ", line)
        line = line.replace("(", "").replace(")", "")
        for x in self.key_words['replace']:
            line = re.sub(r'\b%s\b' % x, ',', line)
        return line

    def get_open_position(self, soup, content):
        open_position = False
        open_term = ""
        text = soup.get_text()
        if contain_keys(text, self.key_words["open_position"], True):
            open_position = True
        if contain_keys(text, self.key_words["open_term"], True):
            open_term = "always"
        return open_position, open_term
    
    def extract_from_sibling(self, node, tags):
        # print node.name, node.string
        next_node = node.find_next_sibling()
        # print next_node
        if next_node and next_node.name == 'ul':
            
            for e in next_node.strings:
                if e and e.strip():
                    tags = self.extract_from_line(e.strip(), tags)
            return tags
        for n in node.find_next_siblings():
            if n.name != next_node.name:
                break
            tags = self.extract_from_line(n.text, tags)
        return tags

    def extract_from_line(self, line, tags):

        for sent in line.split('.'):
            if contain_keys(sent, self.key_words['interest_break'], True):
                break
            sent = self.select_line_part(re.sub("\s+", " ", re.sub("\n", ",", sent)))
            sent = self.replace_words(sent)
            # print "    line    :   ", sent
            for x in re.split("[,:;?]", sent):
                if x and x.strip():
                    tag = x.strip().lower()
                    # print "   tag :  ", tag
                    if ' ' in tag and not contain_keys(tag, tags, True) and tag.count(' ') < 4:
                        tags.append(tag)

        return tags
    

    def get_personal_website(self, l, faculty_url):
        potential_name = re.findall(r"([A-Z]?[a-z]+)", faculty_url) + ['personal']

        potential_name = [e for e in potential_name if not contain_keys(e, self.key_words['keys'] + self.key_words[
            'stop_word'] + self.key_words['notprof'] + ['people', self.university_name])]
        faculty_page = ''
        mail = ''

        for a in l:
            href = a.get('href')
            
            if not href or len(href) < 5:
                continue
                
            suffix = href.split('.')[-1]
            
            if len(suffix) < 5 and contain_keys(suffix, self.key_words['skip_file']):
                continue
            
            if contain_keys(href, potential_name, True) and href not in faculty_url and not onsocial(
                    href):
                if href.startswith('mailto:'):
                    mail = href
                else:
                    faculty_page = href

        return faculty_page, mail
    
    def find_faculty_list(self, l, faculty_url):
        """

        """
        count = 0
        links = []
        faculty_list = []
        # names = find_name_in_emails(l)
        # print names
        for e in l:
            if self.filter_list(e):
                continue

            href = e.get('href')
            faculty_link = format_url(href, self.domain)
            if faculty_link.startswith("Error"):
                print "Error!!!!!! at", href
                continue

            if faculty_link == faculty_url:
                continue

            if faculty_link in links:
                name = e.get_text()
                # print name
                i = links.index(faculty_link)
                if name and not faculty_list[i].string:
                    e['href'] = faculty_link
                    faculty_list[i] = e
                continue
            links.append(faculty_link)
            e['href'] = faculty_link
            faculty_list.append(e)
            count += 1

            name = e.string
            if name is not None and (re.search('Website', name, re.I) or
                                     re.search('personal', name, re.I)):
                # print "personal website:", faculty_link
                pass
            else:
                # print "faculty page:", faculty_link
                pass
        return count, faculty_list
    
    def dive_into_page(self, faculty_ele):
        faculty_link = faculty_ele.get("href")
        person = {'name': '', 'link': faculty_link, 'tags': None,
                'position': False, 'term': ''}
        # print("dive into " + faculty_link)
        if contain_keys(faculty_link.split('/')[-1], self.key_words['skip_file']):
            return person

        if faculty_ele.string and contain_keys(faculty_link, faculty_ele.string.split(), True):
            person['name'] = faculty_ele.string

        content, soup = self.open_page(faculty_link)
        if content.startswith('Error to load'):
            print "Error!!!!!! at the link", faculty_link
            return person
        
        tags = self.get_research_interests(soup, content, [])
        # print("get tags %d ge" % len(tags))
        
        anchors = self.find_all_anchor(soup, self.domain, faculty_link)
        faculty_page, mail = self.get_personal_website(anchors, faculty_link)

        if faculty_page:
            
            # print('website page url %s ' % faculty_page)
            faculty_page = format_url(faculty_page, self.domain, faculty_link)
            page_c, page_soup = self.open_page(faculty_page)
            if page_c.startswith('Error to load'):
                print "Error!!!!!! at the page", faculty_page
            else:
                tags = self.get_research_interests(page_soup, page_c, tags)
                # print("get tags %d ge" % len(tags))
                position, term = self.get_open_position(page_soup, page_c)
                person['position'] = position
                person['term'] = term

        if len(tags) > 1 and tags[0].startswith("I'm so stupid to"):
            tags = tags[1:]
        if len(tags) == 1 and tags[0].startswith("I'm so stupid to"):
            tags = []
        
        person['website'] = faculty_page
        person['mail'] = mail
        person['tags'] = tags

        return person
       
    def get_research_interests(self, soup, content, tags):
        """
        
        """
        # 先用 完整的 research interests 找
        result = soup.find_all(string=re.compile("research\s+interest", re.I))
        # print("research interest has %d " % len(result))
        
        if len(result) == 1:
            # print "      position", result[0].lower().find("interest")
            if len(result[0]) > result[0].lower().find("interest") + 15:
                # print "by line"
                # print "from the line"
                line = self.select_line_part(re.sub("\s+", " ", re.sub("\n", ",", result[0])))
                research_tags = self.extract_from_line(line, tags)
                
                if research_tags:
                    
                    # print(" extract from line %d  ge " % len(research_tags))
                    # print research_tags
                    return research_tags
            node = result[0].parent
            tags = self.extract_from_sibling(node, tags)
            # print(" extract from sibling %d  ge " % len(tags))
            return tags
        elif len(result) > 1:
            for n in result:
                if len(n) > 19:
                    tags = self.extract_from_line(n, tags)
                    # print(" extract from line %d  ge " % len(tags))
                node = n.parent
                tags = self.extract_from_sibling(node, tags)
            return tags
            
        # 再用 research or interests to find
        # then filter it by some rules
        result = soup.find_all(string=re.compile("(research|focuses on|Expertise)", re.I))
        nodes = self.filter_research_interests(result)
        #print("only one has %d "%len(nodes))
        #print tags

        if len(nodes) == 1:
            # print nodes[0]
            if len(nodes[0]) > 19:
                # print "by line",
                research_tags = self.extract_from_line(result[0], tags)
                # print(" extract from line %d  ge " % len(research_tags))
                if research_tags:
                    return research_tags
            node = nodes[0].parent
            # print node.next_sibling
            tags = self.extract_from_sibling(node, tags)
            # print(" extract from sibling %d  ge " % len(tags))
            return tags
        # else 搞不定了
            
        if not len(tags):
            return [u"I'm so stupid to found it,我太蠢了找不到"]
        return tags
    

In [151]:
crawler = ResearchCrawler()
for i in range(6, 7):#len(urls)):
    directory_url = urls[i]
    example = examples[i]
    # print directory_url, example
    c, l = crawler.crawl_faculty_list(directory_url, example)
    for e in l[:5]:
        print l.index(e), e.get('href')
        # print e.get_text()
        x = crawler.dive_into_page(e)

        print x['tags']

NameError: name 'ResearchCrawler' is not defined

In [88]:
for e in l[1:3]:
    href = e.get('href')
    content, soup = crawler.open_page(href)
    

In [28]:
print href

http://science.iit.edu/people/faculty/gady-agam


In [83]:
crawler = ResearchCrawler()
directory_url = urls[1]
example = examples[1]
print directory_url, example
c, l = crawler.crawl_faculty_list(directory_url, example)

http://science.iit.edu/computer-science/people/faculty http://science.iit.edu/people/faculty/eunice-santos
http://science.iit.edu/computer-science/people/faculty 347
http://science.iit.edu/computer-science/people/faculty 168


In [6]:
print len(l), crawler.key_words['stop_word']

52 [u'http', u'https', u'www', u'edu', u'com', u'lab', u'calendar', 'http', 'science', 'iit', 'edu', 'computer', 'science', 'people', 'faculty']


In [131]:
import cProfile
crawler = ResearchCrawler()
for url in urls[8:]:
    print url

    result = crawler.crawl_from_directory(url, examples[urls.index(url)])
    print len(result)
    print result[0]

http://www.cs.ucf.edu/people/index.php
http://www.cs.ucf.edu/people/index.php 651
http://www.cs.ucf.edu/people/index.php 257 [u'http', u'www', u'cs', u'ucf', u'edu', u'bagci', 'personal'] [u'faculty', u'professor', u'lecturer', u'affiliate', u'directory/people', u'http', u'https', u'www', u'edu', u'com', u'lab', u'calendar', 'http', 'www', 'cs', 'ucf', 'edu', 'people', 'index', 'php', u'CONTACTS', u'ADMINISTRATION', u'qa', u'ADVISORY', u'BOARD', u'OPENINGS', 'people', 'ucf']
website page url:  http://www.cs.ucf.edu/~bagci/software.html
[u'http', u'www', u'cs', u'ucf', u'edu', u'csdept', u'faculty', u'bassi', u'html', 'personal'] [u'faculty', u'professor', u'lecturer', u'affiliate', u'directory/people', u'http', u'https', u'www', u'edu', u'com', u'lab', u'calendar', 'http', 'www', 'cs', 'ucf', 'edu', 'people', 'index', 'php', u'CONTACTS', u'ADMINISTRATION', u'qa', u'ADVISORY', u'BOARD', u'OPENINGS', 'people', 'ucf']
[u'http', u'www', u'cs', u'ucf', u'edu', u'lboloni', 'personal'] [u'fac

[u'http', u'www', u'cs', u'ucf', u'edu', u'xiaoman', 'personal'] [u'faculty', u'professor', u'lecturer', u'affiliate', u'directory/people', u'http', u'https', u'www', u'edu', u'com', u'lab', u'calendar', 'http', 'www', 'cs', 'ucf', 'edu', 'people', 'index', 'php', u'CONTACTS', u'ADMINISTRATION', u'qa', u'ADVISORY', u'BOARD', u'OPENINGS', 'people', 'ucf']
website page url:  http://www.cs.ucf.edu/~xiaoman/2012PCB5596BSC4434
[u'http', u'www', u'cs', u'ucf', u'edu', u'feiliu', 'personal'] [u'faculty', u'professor', u'lecturer', u'affiliate', u'directory/people', u'http', u'https', u'www', u'edu', u'com', u'lab', u'calendar', 'http', 'www', 'cs', 'ucf', 'edu', 'people', 'index', 'php', u'CONTACTS', u'ADMINISTRATION', u'qa', u'ADVISORY', u'BOARD', u'OPENINGS', 'people', 'ucf']


KeyboardInterrupt: 

In [23]:

import pstats  
p = pstats.Stats('my_prof')

In [16]:
p.sort_stats('cumtime')
p.print_stats()

Fri Sep 01 10:52:31 2017    my_prof

         7842632 function calls (7836930 primitive calls) in 25.229 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   25.229   25.229 <string>:1(<module>)
        1    0.001    0.001   25.229   25.229 crawler.py:96(crawl_from_directory)
       52    0.003    0.000   24.740    0.476 crawler.py:373(dive_into_page)
       83    0.004    0.000   19.592    0.236 crawler.py:127(open_page)
       89    0.008    0.000   12.502    0.140 E:\software\Python27\lib\site-packages\bs4\__init__.py:87(__init__)
       89    0.001    0.000   11.206    0.126 E:\software\Python27\lib\site-packages\bs4\__init__.py:285(_feed)
       89    0.001    0.000   11.201    0.126 E:\software\Python27\lib\site-packages\bs4\builder\_htmlparser.py:162(feed)
       89    0.001    0.000   11.199    0.126 E:\software\Python27\lib\HTMLParser.py:110(feed)
       89    1.064    0.012   11.198    0

      151    0.001    0.000    0.001    0.000 E:\software\Python27\lib\threading.py:187(release)
      345    0.001    0.000    0.001    0.000 E:\software\Python27\lib\sre_parse.py:268(_escape)
       89    0.000    0.000    0.001    0.000 E:\software\Python27\lib\HTMLParser.py:98(__init__)
  402/278    0.001    0.000    0.001    0.000 E:\software\Python27\lib\site-packages\bs4\element.py:891(string)
       14    0.000    0.000    0.001    0.000 E:\software\Python27\lib\site-packages\bs4\builder\_htmlparser.py:114(unknown_decl)
        2    0.000    0.000    0.001    0.001 E:\software\Python27\lib\linecache.py:13(getline)
       19    0.000    0.000    0.001    0.000 E:\software\Python27\lib\site-packages\chardet\universaldetector.py:81(__init__)
        2    0.000    0.000    0.001    0.001 E:\software\Python27\lib\linecache.py:33(getlines)
        1    0.000    0.000    0.001    0.001 E:\software\Python27\lib\linecache.py:72(updatecache)
        1    0.000    0.000    0.001    0.001 

        1    0.000    0.000    0.000    0.000 E:\software\Python27\lib\site-packages\chardet\euckrprober.py:35(__init__)
        1    0.000    0.000    0.000    0.000 E:\software\Python27\lib\site-packages\chardet\big5prober.py:35(__init__)
        1    0.000    0.000    0.000    0.000 E:\software\Python27\lib\site-packages\chardet\euctwprober.py:34(__init__)
       30    0.000    0.000    0.000    0.000 E:\software\Python27\lib\site-packages\bs4\__init__.py:272(<genexpr>)
      129    0.000    0.000    0.000    0.000 {method 'add' of 'set' objects}
        4    0.000    0.000    0.000    0.000 E:\software\Python27\lib\_abcoll.py:128(__subclasshook__)
        4    0.000    0.000    0.000    0.000 E:\software\Python27\lib\UserDict.py:57(items)
        4    0.000    0.000    0.000    0.000 E:\software\Python27\lib\logging\__init__.py:1071(_fixupParents)
        6    0.000    0.000    0.000    0.000 E:\software\Python27\lib\cookielib.py:1196(deepvalues)
        3    0.000    0.000    0.00

        1    0.000    0.000    0.000    0.000 E:\software\Python27\lib\site-packages\requests\structures.py:62(__len__)
        5    0.000    0.000    0.000    0.000 E:\software\Python27\lib\site-packages\requests\_internal_utils.py:14(to_native_string)
        2    0.000    0.000    0.000    0.000 {method 'format' of 'str' objects}
        1    0.000    0.000    0.000    0.000 {method 'encode' of 'str' objects}
        1    0.000    0.000    0.000    0.000 {_codecs.charmap_build}
        1    0.000    0.000    0.000    0.000 E:\software\Python27\lib\urllib.py:1221(unquote)
        3    0.000    0.000    0.000    0.000 E:\software\Python27\lib\collections.py:133(itervalues)
        1    0.000    0.000    0.000    0.000 E:\software\Python27\lib\site-packages\chardet\chardistribution.py:114(__init__)
        1    0.000    0.000    0.000    0.000 E:\software\Python27\lib\site-packages\chardet\chardistribution.py:218(__init__)
        1    0.000    0.000    0.000    0.000 E:\software\Pytho

<pstats.Stats instance at 0x0000000007E53608>